In [5]:
!python ./code1/src/Data_process.py

In [8]:
!pip install mpi4py

Defaulting to user installation because normal site-packages is not writeable
  Using cached mpi4py-3.1.3.tar.gz (2.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for mpi4py (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [148 lines of output]
      running bdist_wheel
      running build
      running build_src
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-39
      creating build/lib.linux-x86_64-cpython-39/mpi4py
      copying src/mpi4py/__init__.py -> build/lib.linux-x86_64-cpython-39/mpi4py
      copying src/mpi4py/__main__.py -> build/lib.linux-x86_64-cpython-39/mpi4py
      copying src/mpi4py/bench.py -> build/lib.linux-x86_64-cpython-39/mpi4py
      copying src/mpi4py/run.py -> build/lib.linux-x86_64-cpython-39/mpi4py
      creating build/lib.linux-x86_64-cpython-39/

In [10]:
!NUMEXPR_MAX_THREADS=100 python ./code2/src/calc_exact_law_seq.py

Process information recorded in output_ELcalc_08072022_1755.txt
^C
Traceback (most recent call last):
  File "/home/jeandet/Pauline/./code2/src/calc_exact_law_seq.py", line 1385, in <module>
    data_output.calc_at_scale(indices, exactlaw, div=True)
  File "/home/jeandet/Pauline/./code2/src/calc_exact_law_seq.py", line 886, in calc_at_scale
    ] = self.funcdic[t](exactlaw, *self.argdic[t])[indices[3]]
  File "/home/jeandet/Pauline/./code2/src/calc_exact_law_seq.py", line 983, in flux
    return np.array(list(self.flux_gen(data_1, data_2, data_3, data_4)))
  File "/home/jeandet/Pauline/./code2/src/calc_exact_law_seq.py", line 944, in flux_gen
    yield np.sum(tab)  # np.sum(np.sort(tab.flatten()))
  File "<__array_function__ internals>", line 5, in sum
  File "/opt/conda/lib/python3.9/site-packages/numpy/core/fromnumeric.py", line 2259, in sum
    return _wrapreduction(a, np.add, 'sum', axis, dtype, out, keepdims=keepdims,
  File "/opt/conda/lib/python3.9/site-packages/numpy/core/fromn

In [6]:
!rm -rf save_08072022_1743

In [1]:
import numpy as np
import numexpr as ne

In [25]:
data[[1,2,3]].flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [17]:
data[10:].shape

(502, 512, 512)

In [27]:
data.flatten?

Docstring:
a.flatten(order='C')

Return a copy of the array collapsed into one dimension.

Parameters
----------
order : {'C', 'F', 'A', 'K'}, optional
    'C' means to flatten in row-major (C-style) order.
    'F' means to flatten in column-major (Fortran-
    style) order. 'A' means to flatten in column-major
    order if `a` is Fortran *contiguous* in memory,
    row-major order otherwise. 'K' means to flatten
    `a` in the order the elements occur in memory.
    The default is 'C'.

Returns
-------
y : ndarray
    A copy of the input array, flattened to one dimension.

See Also
--------
ravel : Return a flattened array.
flat : A 1-D flat iterator over the array.

Examples
--------
>>> a = np.array([[1,2], [3,4]])
>>> a.flatten()
array([1, 2, 3, 4])
>>> a.flatten('F')
array([1, 3, 2, 4])
Type:      builtin_function_or_method


In [29]:
%timeit a=data.reshape((-1,))

216 ns ± 2.46 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [127]:
data = np.random.random((512,512,512))
a=data.reshape((-1,))
b=np.roll(a,10,0)

In [124]:
%timeit b=np.roll(a,10,0)

2.24 ms ± 21.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [113]:
%timeit np.sum(a[10:] - a[:-10])

2.94 ms ± 34.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [114]:
%timeit np.sum(a - b)

3.63 ms ± 77.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [115]:
%timeit np.sum(a[10:] - a[:-10])

2.87 ms ± 47.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [116]:
%timeit np.add.reduce(a[10:] - a[:-10])

2.84 ms ± 21.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
from numba import njit, set_num_threads
from math import sqrt

@njit('double(double,double)')
def inner_test_f(a, b):
    return sqrt(a - b + (a - b) * (a + b))

@njit('double(double,double,double)')
def inner_test_f2(i,j,a):
    return sqrt(a[i] - a[j] + (a[i] - a[j]) * (a[i] + a[j]))

@njit
def test_f(a, b):
    acc = 0
    for i in range(len(a)):
        acc += inner_test_f( a[i], b[i])
    return acc

@njit
def test_f2(a, dist:int):
    acc = 0
    l = len(a)
    for i in range(l - dist):
        acc +=  inner_test_f(a[i], a[(i+dist)]) 
    offset = l-dist
    for i in range(dist):
        acc +=   inner_test_f(a[i+offset], a[i])
    return acc

@njit
def test_f23(a, dist:int):
    acc = 0
    l = len(a)
    for i in range(l - dist):
        acc +=  inner_test_f2([i], [(i+dist)], a) 
    offset = l-dist
    for i in range(dist):
        acc +=   inner_test_f(a[i+offset], a[i])
    return acc


@njit
def test_f4(a, dist:int, f):
    acc = 0
    l = len(a)
    for i in range(l - dist):
        acc +=  f(a[i], a[(i+dist)]) 
    offset = l-dist
    for i in range(dist):
        acc +=   f(a[i+offset], a[i])
    return acc

@njit
def test_f3(a, dist:int):
    acc = 0
    l = int(len(a))
    for i in range(l):
        acc +=  a[i]- a[int((i+dist)) and int(l-1)]  
    return acc

set_num_threads(1)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(float64, float64)
 
There are 22 candidate implementations:
[1m      - Of which 22 did not match due to:
      Overload of function 'getitem': File: <numerous>: Line N/A.
        With argument(s): '(float64, float64)':[0m
[1m       No match.[0m
[0m
[0m[1mDuring: typing of intrinsic-call at /tmp/ipykernel_26941/2190005536.py (10)[0m
[1m
File "../ipykernel_26941/2190005536.py", line 10:[0m
[1m<source missing, REPL/exec in use?>[0m


In [135]:
%timeit test_f(a, b)

344 ms ± 49.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [136]:
%timeit test_f(a, np.roll(a,10))

460 ms ± 449 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [137]:
%timeit test_f(a[:-10], a[10:]);test_f(a[-10:], a[:10])

321 ms ± 1.01 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [157]:
%timeit test_f2(a, int(10))

360 ms ± 46.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [156]:
%timeit test_f4(a, int(10),inner_test_f)

355 ms ± 63.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [139]:
%timeit test_f3(a, int(10))

117 ms ± 490 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [140]:
ne.set_num_threads(4)

4

In [122]:
%timeit ne.evaluate('sum(a-b)')

3.77 ms ± 37.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [123]:
%timeit np.sum(ne.evaluate('sqrt(a - b + (a - b) * (a + b))'))

8.61 ms ± 440 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [85]:
for i in range(100):
    np.sum(ne.evaluate('sqrt(a - b + (a - b) * (a + b))'))

In [47]:
%timeit (flat_data[:-10] - flat_data[10:]).sum()

200 ms ± 576 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
#indexes = (np.arange(128*128*128, dtype=np.int32)+10)%128
flat_data = data.reshape(-1)

In [30]:
test = flat_data[:-10]

In [ ]:
for v i vectors:
    for law in laws:
        for t in law:
            apply(v,t)

In [57]:
a=flat_data[:-10] 
b=flat_data[10:]
ne.set_num_threads(4)

4

In [42]:
a.nbytes/1024/1024

1023.9999237060547

In [58]:
%timeit ne.evaluate('sum((a*a - b*b)/(a*a + b*b))', casting='same_kind')

494 ms ± 824 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
import cupy as cp

In [3]:
data = cp.array(np.random.random((1024*1024*1024)))

In [4]:
cp.sum(cp.multiply(data , data) - cp.roll(data,10,0))

array(-1.78951678e+08)

In [7]:
data = np.random.random((1024*1024*1024))

In [8]:
np.sum(data - np.roll(data,10,0))

-1.1965983759409937e-12

In [22]:
!ls  /etc/jupyter

jupyter_notebook_config.py  jupyter_server_config.py


In [23]:
!cat /etc/jupyter/jupyter_notebook_config.py

# Copyright (c) Jupyter Development Team.
# Distributed under the terms of the Modified BSD License.
# mypy: ignore-errors
import os
import stat
import subprocess

from jupyter_core.paths import jupyter_data_dir

c = get_config()  # noqa: F821
c.NotebookApp.ip = "0.0.0.0"
c.NotebookApp.port = 8888
c.NotebookApp.open_browser = False

# https://github.com/jupyter/notebook/issues/3130
c.FileContentsManager.delete_to_trash = False

# Generate a self-signed certificate
OPENSSL_CONFIG = """\
[req]
distinguished_name = req_distinguished_name
[req_distinguished_name]
"""
if "GEN_CERT" in os.environ:
    dir_name = jupyter_data_dir()
    pem_file = os.path.join(dir_name, "notebook.pem")
    os.makedirs(dir_name, exist_ok=True)

    # Generate an openssl.cnf file to set the distinguished name
    cnf_file = os.path.join(os.getenv("CONDA_DIR", "/usr/lib"), "ssl", "openssl.cnf")
    if not os.path.isfile(cnf_file):
        with open(cnf_file, "w") as fh:
            fh.write(OPENSSL_CONFIG)

    #

In [ ]:
ne.evaluate('(A - B) * (C - D) ', A=.., C=...)

In [145]:
tttt=set( [ (i,i*100,i*1000) for i in range(31*31*100)] )

In [148]:
ttt= [ (i,i*100,i*1000) for i in range(31*31*100)] 

In [146]:
tttt

{(78791, 7879100, 78791000),
 (92244, 9224400, 92244000),
 (62747, 6274700, 62747000),
 (76200, 7620000, 76200000),
 (53074, 5307400, 53074000),
 (60156, 6015600, 60156000),
 (86321, 8632100, 86321000),
 (37030, 3703000, 37030000),
 (83730, 8373000, 83730000),
 (1387, 138700, 1387000),
 (60604, 6060400, 60604000),
 (44560, 4456000, 44560000),
 (41969, 4196900, 41969000),
 (52810, 5281000, 52810000),
 (29684, 2968400, 29684000),
 (75961, 7596100, 75961000),
 (95983, 9598300, 95983000),
 (27093, 2709300, 27093000),
 (79939, 7993900, 79939000),
 (11049, 1104900, 11049000),
 (18579, 1857900, 18579000),
 (2535, 253500, 2535000),
 (52571, 5257100, 52571000),
 (42898, 4289800, 42898000),
 (26854, 2685400, 26854000),
 (93153, 9315300, 93153000),
 (28241, 2824100, 28241000),
 (34384, 3438400, 34384000),
 (18340, 1834000, 18340000),
 (78277, 7827700, 78277000),
 (62233, 6223300, 62233000),
 (69763, 6976300, 69763000),
 (873, 87300, 873000),
 (83216, 8321600, 83216000),
 (53719, 5371900, 53719000

In [153]:
%timeit ttt[100] in tttt

82.6 ns ± 0.544 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [152]:
%timeit ttt[100] in ttt

1.27 µs ± 2.48 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
